In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install lxml

In [2]:
from bs4 import BeautifulSoup
import lxml
import requests as r
import json
from datetime import date
from datetime import timedelta
import json

In [3]:
#get links of all categories
def getCateLinks():
    main_url = 'http://edition.cnn.com/'
    res = r.get(main_url)
    res.encoding='unicode'
    soup = BeautifulSoup(res.text,'lxml')
    tags = soup.select('.m-footer__bucket_group .m-footer__link')
    
    urlList = []
    
    for tag in tags:
        
        #避開例外 http 開頭連結
        if (tag['href'].lstrip('/').split('/')[0] != 'http:'):
            urlList.append('{}'.format(main_url+tag['href'].lstrip('/')))
    
    return urlList

In [5]:
getCateLinks()[0]

'http://edition.cnn.com/us'

In [8]:
#get all links of articles in each category
def getArticleList(url):
    
    articleList = []

    res = r.get(url)
    res.encoding='unicode'
    soup = BeautifulSoup(res.text,'lxml')
    titles = soup.select('.cd__headline a')
    
    for num in range(len(titles)):
        article_info = {}
        article_info['title'] = titles[num].text
        
        if (titles[num]['href'].lstrip('/').split('/')[0] != 'http:'):
            article_info['url'] = 'http://edition.cnn.com/' + titles[num]['href'].lstrip('/')
        else:
            article_info['url'] = titles[num]['href'].lstrip('/')
        
        articleList.append(article_info)
        
    return articleList

In [12]:
test = []

for i in range(0,2):
    test.append(getArticleList(getCateLinks()[0])[i])
    
print(test)

[{'title': 'Scheduled to die in Arkansas: Does mental competence matter? ', 'url': 'http://edition.cnn.com/2017/04/14/health/arkansas-executions-mental-competence-eprise/index.html'}, {'title': 'Federal judge issues injunction to halt Arkansas executions ', 'url': 'http://edition.cnn.com/2017/04/15/us/arkansas-executions-halted/index.html'}]


In [22]:
def addContent(articleList):
    
    listIncludeContent = []    #output list
    
    for num in range(len(articleList)):
        
        res = r.get(articleList[num]['url'])
        res.encoding='unicode'
        soup = BeautifulSoup(res.text,'lxml')
                
        contents = soup.select('#body-text .zn-body__paragraph')
        
        paragraph_dict = {} 
        for numPara in range(len(contents)):
            paragraph_dict[numPara+1] = contents[numPara].text.lstrip('•')
        
        article_dict = {}    #store hole article details
        article_dict['title'] = articleList[num]['title']
        article_dict['date'] = soup.find(itemprop = 'datePublished')['content'].split('T')[0]
        article_dict['source'] = 'CNN'
        article_dict['url'] = articleList[num]['url']
        article_dict['content'] = paragraph_dict
        
        listIncludeContent.append(article_dict)
            
    return listIncludeContent

In [21]:
addContent(test)

[{'content': {1: ' (CNN)Six of the eight men Arkansas planned to execute over an 11-day period this month are not mentally fit for the death penalty, a report by a Harvard Law School initiative suggests.',
   2: 'The report by the school\'s Fair Punishment Project says that between mental illnesses, intellectual disabilities and "bad lawyering," the orders for all eight of the executions may never have been justified.',
   3: '"Nobody is questioning whether people who commit some of the most serious offenses should be held accountable," said Rob Smith, executive director of the project, which seeks to create a fair and accountable justice system.',
   4: '"The question is, do people who are among the most broken, vulnerable and impaired in our society deserve the death penalty," he said, as opposed to life without parole. ',
   5: "Two of the eight were granted stays on their executions, but six of the men's fates are still being fought in the courts. ",
   6: 'A federal judge issued a

In [15]:
def jsonOut(targetList):
    day = date.today().strftime('%Y%m%d')
    with open('{}_CNNoutput.json'.format(day), 'w') as f:
        json.dump(targetList, f)
        f.close()
    print('File {}_CNNoutput.json has been created!'.format(day))

In [16]:
jsonOut(addContent(test))

File 20170416_CNNoutput.json has been created!
